This notebook is intended to demonstrate how select registration, segmentation, and image mathematical methods of ITKTubeTK can be combined to perform multi-channel brain extraction (aka. skull stripping for patient data containing multiple MRI sequences).

There are many other (probably more effective) brain extraction methods available as open-source software such as BET and BET2 in the FSL package (albeit such methods are only for single channel data).   If you need to perform brain extraction for a large collection of scans that do not contain major pathologies, please use one of those packages.   This notebook is meant to show off the capabilities of specific ITKTubeTK methods, not to demonstration how to "solve" brain extraction.

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

ModuleNotFoundError: No module named 'itkwidgets'

In [2]:
ImageType = itk.Image[itk.F, 3]

ReaderType = itk.ImageFileReader[ImageType]

InputBaseName = "G:/My Drive/Projects/Proj_UNC_StrokeCollaterals/Experiments/UNC/CTP/CTAT-001-MinMax/max"

filename = InputBaseName + ".nrrd"
reader1 = ReaderType.New(FileName=filename)
reader1.Update()
im1 = reader1.GetOutput()

resamp = ttk.ResampleImage[ImageType].New(Input = im1)
resamp.SetMakeHighResIso(True)
resamp.Update()
im1iso = resamp.GetOutput()

filename = InputBaseName + "-Iso.nrrd"
itk.imwrite(im1iso, filename)

ImportError: PyCapsule_Import could not import module "_ITKCommonPython"

In [3]:
N = 8
readerList = ["003", "010", "026", "034", "045", "056", "063", "071"]

imBase = []
imBaseB = []
for i in range(0,N):
    name = "../Data/Normal"+readerList[i]+"-FLASH.mha"
    nameB = "../Data/Normal"+readerList[i]+"-FLASH-Brain.mha"
    reader = ReaderType.New(FileName=name)
    reader.Update()
    imMathNorm = ttk.ImageMath.New(Input=reader.GetOutput())
    imMathNorm.NormalizeMeanStdDev()
    imBaseTmp = imMathNorm.GetOutput()
    reader = ReaderType.New(FileName=nameB)
    reader.Update()
    imBaseBTmp = reader.GetOutput()
    imBase.append(imBaseTmp)
    imBaseB.append(imBaseBTmp)

In [4]:
view(im1iso)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [5]:
thresh = ttk.ImageMath.New(Input=im1iso)
thresh.ReplaceValuesOutsideMaskRange(im1iso,-150,600,-200) #-150,100,-200 for min
thresh.Threshold(-199,600,1,0)
#thresh.NormalizeMeanStdDev()
im1isoT = thresh.GetOutput()
im1iso = im1isoT
view(im1iso)
#itk.imwrite(im1iso,"im1iso.mha")

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [26]:
maskMath = ttk.ImageMath.New(Input=im1iso)
#maskMath.Threshold(-2,2,1,0)
maskMath.Dilate(20,1,0)
maskMathD = maskMath.GetOutput()
maskMath.SetInput(im1iso)
maskMath.Erode(10,1,0)
maskMath.AddImages(maskMathD,-1,1)
mask = maskMath.GetOutput()
#itk.imwrite(mask, "mask.mha")
maskObject = itk.ImageSpatialObject[3,itk.F].New(Image=mask)

In [27]:
maskMath.SetInput(im1iso)
maskMath.Blur(3)
im1isoBlur = maskMath.GetOutput()
#itk.imwrite(im1isoBlur,"im1isoBlur.mha")

In [ ]:
RegisterImagesType = ttk.RegisterImages[ImageType]
regB = []
regBB = []
for i in range(0,N):
    maskMath.SetInput(imBase[i])
    maskMath.Blur(3)
    movingIm = maskMath.GetOutput()
    #itk.imwrite(movingIm,"movingIm.mha")
    regBTo1 = RegisterImagesType.New(FixedImage=im1isoBlur, MovingImage=movingIm)
    regBTo1.SetReportProgress(True)
    regBTo1.SetExpectedOffsetMagnitude(40)
    regBTo1.SetExpectedRotationMagnitude(0.01)
    regBTo1.SetExpectedScaleMagnitude(0.1)
    regBTo1.SetRigidMaxIterations(500)
    regBTo1.SetAffineMaxIterations(500)
    regBTo1.SetRigidSamplingRatio(0.1)
    regBTo1.SetAffineSamplingRatio(0.1)
    regBTo1.SetInitialMethodEnum("INIT_WITH_IMAGE_CENTERS")
    regBTo1.SetFixedImageMaskObject(maskObject)
    regBTo1.SetUseFixedImageMaskObject(True)
    regBTo1.SetRegistration("PIPELINE_AFFINE")
    #regBTo1.SetMetric("MATTES_MI_METRIC")
    #regBTo1.SetMetric("NORMALIZED_CORRELATION_METRIC") - Really slow!
    regBTo1.SetMetric("MEAN_SQUARED_ERROR_METRIC")
    regBTo1.Update()
    img = regBTo1.ResampleImage()
    regB.append( img )
    img = regBTo1.ResampleImage("LINEAR", imBaseB[i])
    regBB.append( img )

In [29]:
regBBT = []
for i in range(0,N):
    imMath = ttk.ImageMath[ImageType,ImageType].New( Input=regBB[i] )
    imMath.Threshold(0,1,0,1)
    img = imMath.GetOutput()
    if i==0:
        imMathSum = ttk.ImageMath[ImageType,ImageType].New( img )
        imMathSum.AddImages( img, 1.0/N, 0 )
        sumBBT = imMathSum.GetOutput()
    else:
        imMathSum = ttk.ImageMath[ImageType,ImageType].New( sumBBT )
        imMathSum.AddImages( img, 1, 1.0/N )
        sumBBT = imMathSum.GetOutput()
        
view(sumBBT)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [ ]:
insideMath = ttk.ImageMath[ImageType,ImageType].New( Input = sumBBT )
insideMath.Threshold(0,0.95,0,1)
insideMath.Dilate(5,1,0)
insideMath.Erode(25,1,0)
brainInside = insideMath.GetOutput()

outsideMath = ttk.ImageMath[ImageType,ImageType].New( Input = sumBBT )
outsideMath.Threshold(0,0,1,0)
outsideMath.Erode(1,1,0)
brainOutsideAll = outsideMath.GetOutput()
outsideMath.Erode(20,1,0)
outsideMath.AddImages(brainOutsideAll, -1, 1)
brainOutside = outsideMath.GetOutput()

outsideMath.AddImages(brainInside,1,2)
brainCombinedMask = outsideMath.GetOutputUChar()

outsideMath.AddImages(im1iso, 10, 1)
brainCombinedMaskView = outsideMath.GetOutput()

In [ ]:
view(brainCombinedMaskView)

In [ ]:
LabelMapType = itk.Image[itk.UC,3]

segmenter = ttk.SegmentConnectedComponentsUsingParzenPDFs[ImageType,LabelMapType].New()
segmenter.SetFeatureImage( im1iso )
segmenter.SetInputLabelMap( brainCombinedMask )
segmenter.SetObjectId( 2 )
segmenter.AddObjectId( 1 )
segmenter.SetVoidId( 0 )
segmenter.SetErodeDilateRadius( 10 )
segmenter.SetHoleFillIterations( 40 )
segmenter.Update()
segmenter.ClassifyImages()
brainCombinedMaskClassified = segmenter.GetOutputLabelMap()


In [ ]:
view(brainCombinedMaskClassified)

In [ ]:
cast = itk.CastImageFilter[LabelMapType, ImageType].New()
cast.SetInput(brainCombinedMaskClassified)
cast.Update()
brainMaskF = cast.GetOutput()

brainMath = ttk.ImageMath[ImageType,ImageType].New(Input = brainMaskF)
brainMath.Threshold(2,2,1,0)
#brainMath.Dilate(1,1,0)
brainMaskD = brainMath.GetOutput()
brainMath.SetInput( im1iso )
brainMath.ReplaceValuesOutsideMaskRange( brainMaskD, 1, 1, 0)
brain = brainMath.GetOutput()

In [ ]:
view(brain)

In [ ]:
writer = itk.ImageFileWriter[ImageType].New(Input = brain)
filename = InputBaseName + "-Brain.nrrd"
writer.SetFileName(filename)
writer.SetUseCompression(True)
writer.Update()